In [69]:
import calendar
import os
import pandas as pd
import sys
from datetime import datetime
from sqlalchemy.sql import extract
from collections import defaultdict, OrderedDict
from sqlalchemy import create_engine
caminho_commons = os.path.join('..', '..', 'ajna_docs', 'commons')
caminho_virasana = os.path.join('..', '..', 'virasana')
sys.path.append(caminho_commons)
sys.path.append('..')
sys.path.append(caminho_virasana)
from ajna_commons.flask.conf import DATABASE, MONGODB_URI
from bhadrasana.models import engine, db_session
from bhadrasana.models import Usuario
from bhadrasana.models.ovr import OVR, EventoOVR
from bhadrasana.models.rvfmanager import lista_rvfovr

In [70]:
def get_fichas_portipo(ano, mes):
    lista_ovrs = db_session.query(OVR).join(EventoOVR)\
                                      .filter(OVR.setor_id.in_((1, 2, 3)))\
                                      .filter(EventoOVR.fase >= 3)\
                                      .filter(extract('year', EventoOVR.create_date) ==  ano)\
                                      .filter(extract('month', EventoOVR.create_date) ==  mes)\
                                      .order_by(OVR.tipooperacao, OVR.id).all()
    lista_fichas = set()
    for ovr in lista_ovrs:
        lista_fichas.add(ovr)
    return lista_fichas

In [71]:
def preenche_linha_ovr(operacao, year, month):
    lastday = calendar.monthrange(year, month)[1]
    ano = str(year)
    mes = str(month).zfill(2)
    inicio = f'01/{mes}/{ano}'
    termino = f'{lastday}/{mes}/{ano}'
    campos_ovr = OrderedDict()
    NOMES_TIPOOPERACAO = {
        1: f'IMPORTAÇÃO - OPERAÇÃO LUNETA {ano}{mes}',
        # Observação: ideal seria dividir mais as OVRs, pois além de Operação Luneta, tivemos Outlet, Ramenta, temos a seleção de 
        # bagagem, etc. Mas aí precisaria ser "Hard-coded" o agrupamento por OVRs
        2: f'EXPORTAÇÃO - OPERAÇÃO PORTO BLINDADO {ano}{mes}',
        # Observação: hoje não temos, mas se fizermos OUTROS TIPOS DE OPERAÇÕES em exportação que não a habitual busca de cocaína,
        # o ideal seria separar em outras OVRs
        3: f'IMPORTAÇÃO - ANÁLISE DE RISCO E DE IMAGENS - OPERAÇÃO LUNETA {ano}{mes}',
        # Observação: ideal seria dividir mais as OVRs, pois além de Operação Luneta, tivemos Outlet, Ramenta, temos a seleção de 
        # bagagem, etc. Mas aí precisaria ser "Hard-coded" o agrupamento por OVRs
        4: f'EXPORTAÇÃO - ANÁLISE DE RISCO E DE IMAGENS - OPERAÇÃO PORTO BLINDADO {ano}{mes}',
        # Observação: existe também o código "Pós-incidente" e não temos distinção no Fichas, vamos criar um Flag.
        # no caso deste flag, substituir estes campos por 'Tipo': 'Pós-Incidente - Portos', 'Código': '30.02.10'
        7: f'VIGILÂNCIA - OPERAÇÃO REDOMA {ano}{mes}',
    }
    campos_ovr['Nome'] = NOMES_TIPOOPERACAO[operacao.tipooperacao]
    if operacao.tipooperacao in (1, 2):
        campos_ovr['Modalidade'] = 'Vigilância e Repressão'
        campos_ovr['Tipo'] = 'Portos'
        campos_ovr['Código'] = '20.00.10'
    elif operacao.tipooperacao in (3, 4):
        campos_ovr['Modalidade'] = 'Pesquisa e Seleção'
        campos_ovr['Tipo'] = 'Pré-Incidente - Portos'
        campos_ovr['Código'] = '30.01.10'
    elif operacao.tipooperacao == 7:
        campos_ovr['Modalidade'] = 'Vigilância e Repressão'
        # Observação: Secta e-OVR não possui a tipificação de vigilância/rondas, auditorias e conformidade
        # Utilizando este tipo e código para distinção
        campos_ovr['Tipo'] = 'Ponto de fronteira'
        campos_ovr['Código'] = '20.00.09'
    else:
        raise Exception(f'tipooperacao {operacao.tipooperacao} não previsto!!')
    campos_ovr['Abrangência'] = 'Local'
    campos_ovr['Início'] = inicio
    campos_ovr['Término'] = termino
    campos_ovr['Local da Execução'] = 'Porto de Santos'
    campos_ovr['Coordenador'] = 'Ivan da Silva Brasílico'
    campos_ovr['Coordenador_CPF'] = '25052288840'
    campos_ovr['Motivação'] = 'Demanda Interna'
    # Por último, analisa Flags para mudar Descrição da Operação
    if len(operacao.flags) > 0:
        flags = [flag for flag in operacao.flags]
        for flag in flags:
            if 'Operação' in flag.nome:
                campos_ovr['Nome'] = flag.nome + f' {ano}{mes}'
                break
            if flag.nome == 'Pós incidente':
                campos_ovr['Nome'] = 'Pós-Incidente - ' + campos_ovr['Nome']
                campos_ovr['Tipo'] = 'Pós-Incidente - Portos'
                campos_ovr['Código'] = '30.02.10'
    return campos_ovr

In [86]:
def get_peso_apreensoes(session, ovr):
    peso = 0.
    rvfs = lista_rvfovr(session, ovr.id)
    for rvf in rvfs:
        for apreensao in rvf.apreensoes:
            try:
                peso += float(apreensao.peso)
            except TypeError:
                pass
    return peso

def get_valor_tgs(ovr):
    valor = 0.
    for tg in ovr.tgs:
        try:
            valor += float(tg.valor)
        except TypeError:
            pass
    return valor


def preenche_linha_operacao(operacao, modalidade):
    linha = OrderedDict()
    linha['Órgão'] = 'RFB'
    linha['Equipe'] = operacao.setor.nome
    linha['Modalidade Operação'] = modalidade
    linha['Tipo'] = 'Portos'
    linha['Local da Operação'] = operacao.recinto.nome
    linha['Documentos de origem'] = f'Ficha {operacao.id}'
    linha['Ocorrências durante a operação'] = operacao.observacoes
    linha['Supervisor da RFB'] = operacao.cpfauditorresponsavel
    linha['Responsável'] = operacao.responsavel_cpf
    servidores = set()
    for evento in operacao.historico:
        # servidor = db_session.query(Usuario).filter(Usuario.cpf == evento.user_name).one()
        servidores.add(evento.user_name)
    linha['Servidores'] = servidores
    linha['Perdimento'] = get_valor_tgs(operacao)
    linha['Apreensao'] = get_peso_apreensoes(db_session, operacao)
    return linha


In [87]:
mes = 3
ano = 2023

lista_fichas = get_fichas_portipo(ano, mes)

In [88]:
lista_final = []
for ficha in lista_fichas:
    try:
        linha_ovr = preenche_linha_ovr(ficha, ano, mes)
        linha_operacao = preenche_linha_operacao(ficha, linha_ovr['Modalidade'])
        linha_final = linha_ovr.copy()
        linha_final.update(linha_operacao)
        lista_final.append(linha_final)
    except Exception as err:
        print(err)
        continue


5
5
6
0
0


In [99]:
df = pd.DataFrame(lista_final)
df = df.sort_values('Nome')

In [100]:
df

,Nome,Modalidade,Tipo,Código,Abrangência,Início,Término,Local da Execução,Coordenador,Coordenador_CPF,...,Equipe,Modalidade Operação,Local da Operação,Documentos de origem,Ocorrências durante a operação,Supervisor da RFB,Responsável,Servidores,Perdimento,Apreensao
0,EXPORTAÇÃO - OPERAÇÃO PORTO BLINDADO 202303,Vigilância e Repressão,Portos,20.00.10,Local,01/03/2023,31/03/2023,Porto de Santos,Ivan da Silva Brasílico,25052288840,...,ALFSTS/DIREP/EQREXP,Vigilância e Repressão,Brasil Terminal Portuario,Ficha 5685,HLBU9485798 selecionado com base na analise da...,13757013808,01126891754,"{00335343813, 00579600629, 01126891754}",0.0,0.0
123,EXPORTAÇÃO - OPERAÇÃO PORTO BLINDADO 202303,Vigilância e Repressão,Portos,20.00.10,Local,01/03/2023,31/03/2023,Porto de Santos,Ivan da Silva Brasílico,25052288840,...,ALFSTS/DIREP/EQREXP,Vigilância e Repressão,SANTOS BRASIL PARTICIPACOES S/A - OPERADOR,Ficha 5622,Seleção feita com base na imagem de escâner\r\...,13757013808,01126891754,"{01126891754, 02768829544, 19281568861}",0.0,0.0
124,EXPORTAÇÃO - OPERAÇÃO PORTO BLINDADO 202303,Vigilância e Repressão,Portos,20.00.10,Local,01/03/2023,31/03/2023,Porto de Santos,Ivan da Silva Brasílico,25052288840,...,ALFSTS/DIREP/EQREXP,Vigilância e Repressão,SANTOS BRASIL PARTICIPACOES S/A - OPERADOR,Ficha 5708,SEKU4720975 selecionado com base na analise da...,13757013808,01126891754,"{01126891754, 00579600629, 59590793800}",0.0,0.0
126,EXPORTAÇÃO - OPERAÇÃO PORTO BLINDADO 202303,Vigilância e Repressão,Portos,20.00.10,Local,01/03/2023,31/03/2023,Porto de Santos,Ivan da Silva Brasílico,25052288840,...,ALFSTS/DIREP/EQREXP,Vigilância e Repressão,ECOPORTO SANTOS - OPERADOR TECONDI,Ficha 5495,GCNU1383139 selecionado com base na analise da...,13757013808,01126891754,"{01126891754, 00579600629, 59590793800}",0.0,0.0
127,EXPORTAÇÃO - OPERAÇÃO PORTO BLINDADO 202303,Vigilância e Repressão,Portos,20.00.10,Local,01/03/2023,31/03/2023,Porto de Santos,Ivan da Silva Brasílico,25052288840,...,ALFSTS/DIREP/EQREXP,Vigilância e Repressão,EMBRAPORT - EMPRESA BRASILEIRA DE TERMINAIS PO...,Ficha 5623,Seleção feita com base na imagem de escâner\r\...,13757013808,01126891754,"{10903789809, 01126891754, 25052288840, 192815...",0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
161,VIGILÂNCIA - OPERAÇÃO REDOMA 202303,Vigilância e Repressão,Portos,20.00.09,Local,01/03/2023,31/03/2023,Porto de Santos,Ivan da Silva Brasílico,25052288840,...,ALFSTS/DIREP/EQREXP,Vigilância e Repressão,ALFANDEGA,Ficha 5862,"Ronda no Canal do Porto, Marinas e Fundeadouro...",None,None,{29260847818},0.0,0.0
165,VIGILÂNCIA - OPERAÇÃO REDOMA 202303,Vigilância e Repressão,Portos,20.00.09,Local,01/03/2023,31/03/2023,Porto de Santos,Ivan da Silva Brasílico,25052288840,...,ALFSTS/DIREP/EQREXP,Vigilância e Repressão,ALFANDEGA,Ficha 5863,"Ronda no Canal do Porto, Marinas e Fundeadouro...",None,None,{29260847818},0.0,0.0
169,VIGILÂNCIA - OPERAÇÃO REDOMA 202303,Vigilância e Repressão,Portos,20.00.09,Local,01/03/2023,31/03/2023,Porto de Santos,Ivan da Silva Brasílico,25052288840,...,ALFSTS/DIREP/EQREXP,Vigilância e Repressão,ALFANDEGA,Ficha 5864,"Ronda no Canal do Porto, Marinas e Fundeadouro...",None,None,{29260847818},0.0,0.0
177,VIGILÂNCIA - OPERAÇÃO REDOMA 202303,Vigilância e Repressão,Portos,20.00.09,Local,01/03/2023,31/03/2023,Porto de Santos,Ivan da Silva Brasílico,25052288840,...,ALFSTS/DIREP/EQREXP,Vigilância e Repressão,ALFANDEGA,Ficha 5867,"Ronda no Canal do Porto, Marinas e Fundeadouro...",None,None,{29260847818},0.0,0.0


In [101]:
df.groupby(df.Nome).agg(
    quantidade=pd.NamedAgg(column='Documentos de origem', aggfunc='count'),
    perdimentos=pd.NamedAgg(column='Perdimento', aggfunc='sum'),
    apreensoes=pd.NamedAgg(column='Apreensao', aggfunc='sum'),
).reset_index()

,Nome,quantidade,perdimentos,apreensoes
0,EXPORTAÇÃO - OPERAÇÃO PORTO BLINDADO 202303,158,0.0,117.83
1,IMPORTAÇÃO - ANÁLISE DE RISCO E DE IMAGENS - O...,7,0.0,0.00
2,IMPORTAÇÃO - OPERAÇÃO LUNETA 202303,15,6488227.0,0.00
3,Operação Outlet 202303,2,1802405.0,0.00
4,VIGILÂNCIA - OPERAÇÃO REDOMA 202303,21,0.0,0.00


In [9]:
df.to_excel('teste.xlsx')